In [1]:
import pandas as pd
import numpy as np
import sklearn
import csv
import re
from textblob import TextBlob
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import numpy

In [2]:
STOP = set(stopwords.words('english'))
words = pd.read_csv('./Data/U_Keyword_Dataset.txt', sep='/n', quoting=csv.QUOTE_NONE,
                    names=["Urgency_Keywords"])  # csv comma seperated value
words = words['Urgency_Keywords'].values  # to numpy
words = words.tolist()  # to list
print(words)

['limited', 'time', 'only', 'urgent', 'hurry', 'going', 'now', 'rush', 'last', 'chance', 'deadline', 'final', 'close', 'out', 'going', 'bussiness', 'day', 'never', 'again', 'clerence', "don't", 'deley', 'not', 'miss', 'expires', 'compelling', 'critical', 'crucial', 'demanding', 'essential', 'immediate', 'imperative', 'important', 'indispensable', 'necessary', 'persuasive', 'pressing', 'serious', 'vital', 'weighty', 'burning', 'called-for', 'capital', 'chief', 'clamant', 'clamorous', 'crying', 'demanded', 'driving', 'exigent', 'foremost', 'heavy', 'hurry-up', 'impelling', 'importunate', 'insistent', 'instant', 'leading', 'death', 'momentous', 'paramount', 'primary', 'principal', 'required', 'salient', 'top-priority', 'touch', 'touchy', 'quick', 'fast', 'wanted', 'ASAP', 'soon', 'possible']


/home/ratan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel package so we can avoid doing imports until


In [3]:
ps =PorterStemmer()
SplitIntoWords = lambda message: TextBlob(message.lower()).words
WordsIntoBaseForm = lambda message: [ps.stem(word) for word in SplitIntoWords(message)]


sub_str = "Its urgently work please reply quick"
sub_str= WordsIntoBaseForm(sub_str)
print(sub_str)

['it', 'urgent', 'work', 'pleas', 'repli', 'quick']


In [4]:
count = 0
for sub_sub_str in sub_str:
    if sub_sub_str in words:
        count +=1
print(count)

2


## Create Filtered data

In [5]:
# Original data link

#https://www.kaggle.com/wcukierski/enron-email-dataset/downloads/emails.csv/2

In [6]:
chunk = pd.read_csv("Data/emails.csv", chunksize=2000)
data = next(chunk)

In [7]:
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 2 columns):
file       2000 non-null object
message    2000 non-null object
dtypes: object(2)
memory usage: 31.3+ KB


,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...


In [8]:
def get_text(Series, row_num_slicer):
    """returns a Series with text sliced from a list split from each message. Row_num_slicer
    tells function where to slice split text to find only the body of the message."""
    result = pd.Series(index=Series.index)
    for row, message in enumerate(Series):
        message_words = message.split('\n')
        del message_words[:row_num_slicer]
        result.iloc[row] = message_words
    return result

def get_row(Series, row_num):
    """returns a single row split out from each message. Row_num is the index of the specific
    row that you want the function to return."""
    result = pd.Series(index=Series.index)
    for row, message in enumerate(Series):
        message_words = message.split('\n')
        message_words = message_words[row_num]
        result.iloc[row] = message_words
    return result

def get_address(df, Series, num_cols=1):
    """returns a specified email address from each row in a Series"""
    address = re.compile('[\w\.-]+@[\w\.-]+\.\w+')
    addresses = []
    result1 = pd.Series(index=df.index)
    result2 = pd.Series(index=df.index)
    result3 = pd.Series(index=df.index)
    for i in range(len(df)):
        for message in Series:
            correspondents = re.findall(address, message)
            addresses.append(correspondents)
            result1[i] = addresses[i][0]
        if num_cols >= 2:
            if len(addresses[i]) >= 3:
                result2[i] = addresses[i][1]
                if num_cols == 3:
                    if len(addresses[i]) >= 4:
                        result3[i] = addresses[i][2]
    return result1, result2, result3

def standard_format(df, Series, string, slicer):
    """Drops rows containing messages without some specified value in the expected locations. 
    Returns original dataframe without these values. Don't forget to reindex after doing this!!!"""
    rows = []
    for row, message in enumerate(Series):
        message_words = message.split('\n')
        if string not in message_words[slicer]:
            rows.append(row)
    df = df.drop(df.index[rows])
    return df

In [9]:
x = len(data.index)
headers = ['Message-ID: ', 'Date: ', 'From: ', 'To: ', 'Subject: ']
for i, v in enumerate(headers):
    data = standard_format(data, data.message, v, i)
data = data.reset_index()
print("Got rid of {} useless emails! That's {}% of the total number of messages in this dataset.".format(x - len(data.index), np.round(((x - len(data.index)) / x) * 100, decimals=2)))

Got rid of 165 useless emails! That's 8.25% of the total number of messages in this dataset.


In [10]:
data['text'] = get_text(data.message, 15)
data['date'] = get_row(data.message, 1)
data['senders'] = get_row(data.message, 2)
data['recipients'] = get_row(data.message, 3)
data['subject'] = get_row(data.message, 4)

data.date = data.date.str.replace('Date: ', '')
data.date = pd.to_datetime(data.date)

data.subject = data.subject.str.replace('Subject: ', '')

data['recipient1'], data['recipient2'], data['recipient3'] = get_address(data, data.recipients, num_cols=3)
data['sender'], x, y = get_address(data, data.senders)

del data['recipients']
del data['senders']
del data['file']
del data['message']

data = data[['date', 'sender', 'recipient1', 'recipient2', 'recipient3', 'subject', 'text']]

print(data.head())

                 date                   sender               recipient1  \
0 2001-05-14 23:39:00  phillip.allen@enron.com     tim.belden@enron.com   
1 2001-05-04 20:51:00  phillip.allen@enron.com  john.lavorato@enron.com   
2 2000-10-18 10:00:00  phillip.allen@enron.com   leah.arsdall@enron.com   
3 2000-10-23 13:13:00  phillip.allen@enron.com    randall.gay@enron.com   
4 2000-08-31 12:07:00  phillip.allen@enron.com     greg.piper@enron.com   

  recipient2  recipient3    subject  \
0        NaN         NaN              
1        NaN         NaN        Re:   
2        NaN         NaN   Re: test   
3        NaN         NaN              
4        NaN         NaN  Re: Hello   

                                                text  
0                      [, Here is our forecast, ,  ]  
1  [, Traveling to have a business meeting takes ...  
2                 [, test successful.  way to go!!!]  
3  [, Randy,, ,  Can you send me a schedule of th...  
4            [, Let's shoot for Tuesday

In [11]:
data.to_csv("Data/newdata.csv")